# Flight Analysis: Teacher "Brain" Evaluation

This notebook analyzes the `telemetry_fresh.csv` dataset generated by the "Professional" Teacher script to objectively measure its performance. We are looking for:
1. **Flight Smoothness**: Is it oscillating? (Yaw rate variance)
2. **Safety**: Did it maintain the stored 8m+ safe distance?
3. **Decisiveness**: Did it maintain speed or keep stopping?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

# Load Data
df = pd.read_csv('../dataset/px4_teacher/telemetry_fresh.csv')
print(f"Loaded {len(df)} frames of flight data.")

# Parse Lidar
# We can just look at lidar_min for now
df['lidar_min'] = pd.to_numeric(df['lidar_min'], errors='coerce')

# Calculate Speed
df['speed'] = np.sqrt(df['vx']**2 + df['vy']**2)
df['yaw_rate'] = df['yaw'].diff() / (df['timestamp'].diff() + 1e-6)

# Filter startup noise
df = df[df['timestamp'] > df['timestamp'].min() + 5]

# Use 'rel_alt' instead of 'relative_alt' based on CSV header
df.describe()[['lidar_min', 'speed', 'rel_alt', 'yaw_rate']]

Loaded 6001 frames of flight data.


,lidar_min,speed,rel_alt,yaw_rate
count,5901.0,5901.000000,5901.000000,5901.000000
mean,20.0,2.012448,14.672328,0.570025
std,0.0,0.289772,1.412420,250.428001
min,20.0,0.172047,3.315000,-7019.386144
25%,20.0,1.989271,14.952001,0.000000
50%,20.0,1.993790,14.992001,0.000000
75%,20.0,2.000100,15.019001,0.176402
max,20.0,5.360783,15.112000,7069.938750


: 

In [ ]:
# Plot Path vs Obstacles (min dist) -> 2D Track
plt.figure(figsize=(10,10))
plt.plot(df['lon'], df['lat'], label='Flight Path')
plt.scatter(df['lon'], df['lat'], c=df['lidar_min'], cmap='RdYlGn', s=5, alpha=0.5)
plt.colorbar(label='Min Obstacle Dist (m)')
plt.title("Flight Path colored by Safety Margin")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.axis('equal')
plt.grid(True)
plt.show()

# Plot Yaw Rate and Consistency
plt.figure(figsize=(12, 4))
plt.plot(df['timestamp'] - df['timestamp'].iloc[0], df['yaw_rate'], label='Yaw Rate (deg/s)')
plt.axhline(0, color='k', linewidth=0.5)
plt.ylim(-60, 60)
plt.title("Yaw Stability (Oscillation Check)")
plt.xlabel("Time (s)")
plt.legend()
plt.show()